In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
from matplotlib.patches import Circle
import os
import pickle
from scipy.stats import mannwhitneyu, ttest_ind, kruskal, f_oneway
from statannotations.Annotator import Annotator
from itertools import combinations
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import clear_output, display

In [3]:
df_food_scaled = pd.read_csv('df_food_scaled.csv')

C:\Users\Sophie\AppData\Local\Temp\ipykernel_9812\4074791732.py:1: DtypeWarning: Columns (1,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_food_scaled = pd.read_csv('df_food_scaled.csv')


In [4]:
nutrient_info = {
    'folate_eaten':                                         {'unit': 'mcg', 'target': 400,  'type': 'qualifying'},
    'niacin_eaten':                                         {'unit': 'mg',  'target': 14,   'type': 'qualifying'},
    'pantothenic_acid_eaten':                               {'unit': 'mg',  'target': 5,    'type': 'qualifying'},
    'vitamin_b2_eaten':                                     {'unit': 'mg',  'target': 1.1,  'type': 'qualifying'},
    'vitamin_b1_eaten':                                     {'unit': 'mg',  'target': 1.2,  'type': 'qualifying'},
    'all_trans_retinol_equivalents_activity_eaten':         {'unit': 'IU',  'target': 700,  'type': 'qualifying'},
    'beta_carotene_eaten':                                  {'unit' : 'mcg','target' : 700, 'type' :'qualifying'},
    'vitamin_b12_eaten':                                    {'unit': 'mcg', 'target': 2.4,  'type': 'qualifying'},
    'vitamin_b6_eaten':                                     {'unit': 'mg',  'target': 1.3,  'type': 'qualifying'},
    'vitamin_c_eaten':                                      {'unit': 'mg',  'target': 75,   'type': 'qualifying'},
    'vitamin_d_eaten':                                      {'unit': 'mcg', 'target': 7.5,  'type': 'qualifying'},
    'vitamin_e_activity_eaten':                             {'unit': 'TAE', 'target': 15,    'type': 'qualifying'},
    'calcium_eaten':                                        {'unit': 'mg',  'target': 1000, 'type': 'qualifying'},
    'iron_eaten':                                           {'unit': 'mg',  'target': 18,   'type': 'qualifying'},
    'magnesium_eaten':                                      {'unit': 'mg',  'target': 320,  'type': 'qualifying'},
    'phosphorus_eaten':                                     {'unit': 'mg',  'target': 700,  'type': 'qualifying'},
    'potassium_eaten':                                      {'unit': 'mg',  'target': 4700, 'type': 'qualifying'},
    'zinc_eaten':                                           {'unit': 'mg',  'target': 8,    'type': 'qualifying'},
    'fiber_eaten':                                          {'unit': 'g',   'target': 25,   'type': 'qualifying'},
    'protein_eaten':                                        {'unit': 'g',   'target': 46,   'type': 'qualifying'},
    'fat_eaten':                                            {'unit': 'g',   'target': 78,   'type': 'disqualifying'},
    'fatty_acids_saturated_eaten':                          {'unit': 'g',   'target': 22,   'type': 'disqualifying'},
    'cholesterol_eaten':                                    {'unit': 'mg',  'target': 300,  'type': 'disqualifying'},
    'sugar_eaten':                                          {'unit': 'g',   'target': 125,  'type': 'disqualifying'},
    'sodium_eaten':                                         {'unit': 'mg',  'target': 2400, 'type': 'disqualifying'},
    'salt_eaten' :                                          {'unit' : 'g',  'target': 6,   'type': 'disqualifying'}
}


conversion_factors = {
    "mg": 1000,
    "g": 1,
    "mcg": 1000000,
    "IU": 1,
    "TAE": 1,
}

In [5]:
qualifying_nutrients = {nutr: info for nutr, info in nutrient_info.items() if info['type'] == 'qualifying'}
disqualifying_nutrients = {nutr: info for nutr, info in nutrient_info.items() if info['type'] == 'disqualifying'}

In [6]:
def compute_index(row, nutrient_cols, scaling_factor=2000) :

    index = 0
    ratio_sum = 0

    for nutr in nutrient_cols :
        ratio_sum += row[nutr + '_ratio']

    index = (scaling_factor / row['energy_kcal_eaten']) * (ratio_sum / len(nutrient_cols))
    return index

In [7]:
def compute_qi_excluding(row, nutrient_list, exclude=None, scaling_factor=2000):
    if exclude is not None:
        new_list = [nutr for nutr in nutrient_list if nutr != exclude]
    else:
        new_list = nutrient_list
    

    return compute_index(row, new_list, scaling_factor=scaling_factor)

In [8]:
df_pulse = df_food_scaled[df_food_scaled['food_group'] == 'pulses']
df_pulse.drop_duplicates('combined_name')[['combined_name', 'QI', 'DI', 'NB']].sort_values('QI', ascending=False)

,combined_name,QI,DI,NB
269120,Lentils brown (beige),60.585566,0.049408,61.563855
284412,Farmer's Best Special - Edamame,5.761437,1.467169,85.000000
291266,Edamame with sea salt,2.290742,6.224116,79.542773
281263,Compal - Haricot Noir,2.224240,0.764482,70.413534
293126,Lentilles vertes Casino Bio,2.162836,0.822315,73.266156
...,...,...,...,...
307222,MClassic - Hummus nature,0.107881,0.756180,9.692892
310513,Nature Active Bio - Bio Falafel (original),0.083031,0.497129,5.000000
316340,Florentin - Bio Hummus,0.079815,0.701988,5.000000
312215,Al'fez Authentic FALAFEL MIX WITH CHICK PEAS,0.065680,0.684894,5.000000


In [9]:
df_pulse[df_pulse['combined_name'] == 'Lentils brown (beige)'].drop_duplicates('combined_name')[[i for i in df_pulse.columns if '_ratio_scaled' in i]].T

,269120
folate_eaten_ratio_scaled,1.766393
niacin_eaten_ratio_scaled,0.667447
pantothenic_acid_eaten_ratio_scaled,1.147541
vitamin_b2_eaten_ratio_scaled,0.000000
vitamin_b1_eaten_ratio_scaled,1188.524590
all_trans_retinol_equivalents_activity_eaten_ratio_scaled,0.000000
beta_carotene_eaten_ratio_scaled,0.318501
vitamin_b12_eaten_ratio_scaled,0.000000
vitamin_b6_eaten_ratio_scaled,1.639344
vitamin_c_eaten_ratio_scaled,0.000000


In [10]:
row = df_pulse[df_pulse['combined_name'] == 'Lentils brown (beige)'].iloc[0]

qi_excl = compute_qi_excluding(row, qualifying_nutrients, exclude='vitamin_b1_eaten', scaling_factor=2000)
print(qi_excl)


1.2203546616828864


In [11]:
df_pulse[df_pulse['combined_name'] == "Farmer's Best Special - Edamame"].drop_duplicates('combined_name')[[i for i in df_pulse.columns if '_ratio_scaled' in i]].T

,284412
folate_eaten_ratio_scaled,6.764706
niacin_eaten_ratio_scaled,2.605042
pantothenic_acid_eaten_ratio_scaled,9.882353
vitamin_b2_eaten_ratio_scaled,5.347594
vitamin_b1_eaten_ratio_scaled,3.431373
all_trans_retinol_equivalents_activity_eaten_ratio_scaled,0.000000
beta_carotene_eaten_ratio_scaled,26.302521
vitamin_b12_eaten_ratio_scaled,0.000000
vitamin_b6_eaten_ratio_scaled,5.882353
vitamin_c_eaten_ratio_scaled,9.803922


In [12]:
row = df_pulse[df_pulse['combined_name'] == "Farmer's Best Special - Edamame"].iloc[0]

qi_excl = compute_qi_excluding(row, qualifying_nutrients, exclude='beta_carotene_eaten', scaling_factor=2000)
print(qi_excl)

4.680327587173743
